In [7]:
from selenium import webdriver as wd
import chromedriver_binary
from bs4 import BeautifulSoup
import pandas as pd
import requests
import random
import time

url_tablete = "https://www.emag.ro/tablete/brand/samsung/filter/tehnologii-f7980,wi-fi-v-6996724/numar-nuclee-f7983,8-v-4421298/capacitate-stocare-f8881,128-gb-v-4109360/sort-priceasc/c"
url_laptopuri = "https://www.emag.ro/laptopuri/brand/microsoft/filter/tip-laptop-f7882,ultraportabil-v-4670916/tip-laptop-f7882,home-v-3292210/tip-laptop-f7882,business-v-17787/capacitate-memorie-f7886,6-8-gb-v25915/sort-priceasc/c"

## webdriver stuff, opens Chrome, surfs pages. Order processing. Can be used as an autobuy bot. This one will probably not work on your PC just like that. Maybe you have to adjust the waiting timbe and so on. The next section that uses requests() is the really nice part.

In [ ]:
wd = wd.Chrome()

time.sleep(1)

page_url = url_laptopuri
wd.get(page_url)

time.sleep(8)

#Cookie Settings
preferences_settings = wd.find_element_by_xpath('/html/body/div[13]/div/div[2]/button[2]')
preferences_settings.click()
time.sleep(random.uniform(0.9, 1.6))
accept_button = wd.find_element_by_xpath('/html/body/div[14]/div/div/div[2]/form/div[2]/button[2]')
accept_button.click()

time.sleep(random.uniform(1, 3))

#Scroll 
wd.execute_script("window.scrollTo(0, 300)")


#sort by discount
sort_button = wd.find_element_by_xpath('//*[@id="main-container"]/section[1]/div/div[3]/div[2]/div[1]/div[6]/div/div[2]/div[1]/button')
sort_button.click()

time.sleep(random.uniform(2, 4))

sort_by_discount_button = wd.find_element_by_xpath('//*[@id="main-container"]/section[1]/div/div[3]/div[2]/div[1]/div[6]/div/div[2]/div[1]/div/ul/li[6]/a')
sort_by_discount_button.click()

#Scroll
wd.execute_script("window.scrollTo(0, 400)")

time.sleep(1)

In [ ]:
#Get the price
wd.refresh()
time.sleep(random.uniform(2, 5))
product_price = wd.find_element_by_xpath('/html/body/div[4]/div[2]/div/section[1]/div/div[3]/div[2]/div[5]/div[1]/div/div/div[4]/div[1]/p[2]')
print('Pretul produsului cu reducerea cea mai mare este: ', product_price.text)

#Convert price box to number function
def priceToNumber(x):
    price = x
    price = price[0:-4]
    [x,y] = price.split(",")
    full_price = float(x.replace(".",'')) + float(y)/10
    return full_price

#Refresh page and check price 10 times
i=0
while i < 10:
    wd.execute_script("window.scrollTo(400, 1000)")
    wd.execute_script("window.scrollTo(1000, 400)")
    wd.refresh()
    time.sleep(random.uniform(1, 5))
    product_price = wd.find_element_by_xpath('/html/body/div[4]/div[2]/div/section[1]/div/div[3]/div[2]/div[5]/div[1]/div/div/div[4]/div[1]/p[2]')
    if priceToNumber(product_price.text) > 1000:
        print('Pretul produsului cu reducerea cea mai mare este: ', product_price.text, i)
    i += 1
print("The end")

## Scrape with request() to pandas DataFrame. Further saved to csv file. Can be used to retrieve data periodically and analyse trends.

url_laptopuri: https://www.emag.ro/laptopuri/brand/microsoft/filter/tip-laptop-f7882,ultraportabil-v-4670916/tip-laptop-f7882,home-v-3292210/tip-laptop-f7882,business-v-17787/capacitate-memorie-f7886,6-8-gb-v25915/sort-priceasc/c

url_tablete: https://www.emag.ro/tablete/brand/samsung/filter/tehnologii-f7980,wi-fi-v-6996724/numar-nuclee-f7983,8-v-4421298/capacitate-stocare-f8881,128-gb-v-4109360/sort-priceasc/c

In [10]:
#Extracting product data into Pandas DataFrame
import json

#Preparing lunch - BeautifulSoup Stuff
page = requests.get(url_tablete)
soup = BeautifulSoup(page.text, 'html.parser')


keys_to_pop = ['scm_super_category']
products_dict = pd.DataFrame()
i = 0
it = 0
for tag2 in soup.find_all('button'):
    if i>0:
        x = tag2.get('data-product')
        if len(str(x)) == 4:
            continue
        else:
            #print(x, "\n\n","PLM",i,len(str(x)))
            result_dict = dict(json.loads(x))
            result_dict.pop('scm_super_category')
            if it == 0:
                products_dict = pd.DataFrame([result_dict])               
                #print(products_dict)
                it=1
            else:
                products_dict = pd.concat([products_dict,pd.DataFrame([result_dict])])
                products_dict.reindex()
    i+=1
products_dict.reset_index(drop=True, inplace=True)
products_dict

,pnk,productid,offerid,has_family,is_family,product_name,options_modal,currency,price,category_trail
0,DSMV9FMBM,42916386,88448590,True,False,"Tableta Samsung Galaxy Tab A8, Octa-Core, 10.5...",False,RON,1400.00,"Laptop, Tablete & Telefoane/Tablete & accesori..."
1,D2K50VMBM,45575677,94153599,True,False,"Tableta Samsung Galaxy Tab A8, Octa-Core, 10.5...",False,RON,1510.20,"Laptop, Tablete & Telefoane/Tablete & accesori..."
2,DTX9WFMBM,43389560,89596344,False,False,"Tableta Samsung Galaxy Tab A8, Octa-Core, 10.5...",False,RON,1749.99,"Laptop, Tablete & Telefoane/Tablete & accesori..."
3,DP306NMBM,37754912,96714805,True,False,"Tableta Samsung Galaxy Tab S6 Lite, 128GB, 4GB...",False,RON,1899.00,"Laptop, Tablete & Telefoane/Tablete & accesori..."
4,D485FKMBM,36490057,72072951,False,False,"Tableta Samsung Galaxy Tab S6 Lite, Octa-Core,...",False,RON,1975.00,"Laptop, Tablete & Telefoane/Tablete & accesori..."
5,D9DWHJMBM,43762218,90482710,True,False,"Tableta Samsung Galaxy Tab S6 Lite, 128GB, 4GB...",False,RON,1999.99,"Laptop, Tablete & Telefoane/Tablete & accesori..."
6,DGV2BXMBM,40639355,99387094,True,False,"Tableta Samsung Galaxy Tab S6 Lite, Octa-Core,...",False,RON,2041.93,"Laptop, Tablete & Telefoane/Tablete & accesori..."
7,DDRC43MBM,31985744,62116175,True,False,"Tableta Samsung Galaxy Tab S6 Lite, Octa-Core,...",False,RON,2449.00,"Laptop, Tablete & Telefoane/Tablete & accesori..."
8,DBM5X3MBM,31812014,95537702,True,False,"Tableta Samsung Galaxy Tab S7, Octa-Core, 11"",...",False,RON,2499.99,"Laptop, Tablete & Telefoane/Tablete & accesori..."
9,DQJQQXMBM,41373492,99127145,True,False,"Tableta Samsung Galaxy Tab S7 FE, Octa-Core, 1...",False,RON,2599.00,"Laptop, Tablete & Telefoane/Tablete & accesori..."


In [11]:
#Save to XLSX file
wrt = pd.ExcelWriter('tablete_prices.xlsx')
products_dict.to_excel(wrt, 'tablete')
wrt.save()

In [12]:
#Prepare dinner

for tag in soup.find_all('div'):
    x = tag.find_all('p')
    if len(x) != 0:
        break
        
s2 = BeautifulSoup(str(x))
type(s2)

bs4.BeautifulSoup

In [13]:
i=0
price_list = []
for tag in s2.find_all('p'):
    var = tag.text
    price_list.append(var)
    i+=1


## Trying to also get the PRP Price to check for big differences

In [17]:
#Remove unnecessary stuff
for i in price_list:
    print(i)
    if len(i) == 1:
        price_list.remove(i)

stripped_line = [i.replace('\n','') for i in price_list]
stripped_line = [i.replace('    ','') for i in stripped_line]
stripped_line = [i.replace('\xa0',' ') for i in stripped_line]

#Aici mai trebe detectat unde nu e enter, nu exista PRP si se haoseaza dupa aia

stripped_line = stripped_line[:-3]



1.550,00 Lei

1.400,00 Lei
1.510,20 Lei
1.749,99 Lei
1.899,00 Lei

    PRP: 2.134,99 Lei


1.975,00 Lei

    PRP: 2.099,99 Lei


1.999,99 Lei
2.041,93 Lei
2.449,00 Lei
NOU 2.928,00 Lei
de la 2.499,99 Lei
2.599,00 Lei
2.599,00 Lei
2.599,00 Lei

    PRP: 3.699,99 Lei


3.249,00 Lei

    PRP: 3.999,99 Lei


3.499,99 Lei

    PRP: 4.449,99 Lei


3.499,99 Lei

    PRP: 3.699,98 Lei


3.568,81 Lei

    PRP: 3.699,99 Lei


3.568,81 Lei

    PRP: 4.449,99 Lei


3.569,99 Lei

    PRP: 4.562,17 Lei


3.599,00 Lei
4.074,00 Lei
4.124,99 Lei
4.199,00 Lei

    PRP: 4.818,48 Lei


4.199,00 Lei

    PRP: 4.699,99 Lei


4.299,99 Lei
4.479,79 Lei


4.649,01 Lei

4.498,99 Lei
4.499,90 Lei
4.499,90 Lei

    PRP: 4.799,00 Lei


4.554,00 Lei

    PRP: 5.199,99 Lei


4.571,02 Lei
4.800,99 Lei

    PRP: 5.299,00 Lei


4.849,00 Lei

    PRP: 5.699,99 Lei


4.984,99 Lei
5.399,99 Lei

    PRP: 6.449,99 Lei


5.526,93 Lei
6.022,47 Lei
de la 7.096,38 Lei
Vezi produse
Vezi mai mult 
Alti vizitatori au fost intere

In [18]:
stripped_line

['1.550,00 Lei',
 '1.400,00 Lei',
 '1.510,20 Lei',
 '1.749,99 Lei',
 '1.899,00 Lei',
 'PRP: 2.134,99 Lei',
 '1.975,00 Lei',
 'PRP: 2.099,99 Lei',
 '1.999,99 Lei',
 '2.041,93 Lei',
 '2.449,00 Lei',
 'NOU 2.928,00 Lei',
 'de la 2.499,99 Lei',
 '2.599,00 Lei',
 '2.599,00 Lei',
 '2.599,00 Lei',
 'PRP: 3.699,99 Lei',
 '3.249,00 Lei',
 'PRP: 3.999,99 Lei',
 '3.499,99 Lei',
 'PRP: 4.449,99 Lei',
 '3.499,99 Lei',
 'PRP: 3.699,98 Lei',
 '3.568,81 Lei',
 'PRP: 3.699,99 Lei',
 '3.568,81 Lei',
 'PRP: 4.449,99 Lei',
 '3.569,99 Lei',
 'PRP: 4.562,17 Lei',
 '3.599,00 Lei',
 '4.074,00 Lei',
 '4.124,99 Lei',
 '4.199,00 Lei',
 'PRP: 4.818,48 Lei',
 '4.199,00 Lei',
 'PRP: 4.699,99 Lei',
 '4.299,99 Lei',
 '4.479,79 Lei',
 '4.649,01 Lei',
 '4.498,99 Lei',
 '4.499,90 Lei',
 '4.499,90 Lei',
 'PRP: 4.799,00 Lei',
 '4.554,00 Lei',
 'PRP: 5.199,99 Lei',
 '4.571,02 Lei',
 '4.800,99 Lei',
 'PRP: 5.299,00 Lei',
 '4.849,00 Lei',
 'PRP: 5.699,99 Lei',
 '4.984,99 Lei',
 '5.399,99 Lei',
 'PRP: 6.449,99 Lei',
 '5.526,9